In [1]:
from numbers import Number
import numpy as np
import matplotlib.pyplot as plt
import onix.models.hyperfine as hyperfine

In [15]:
ground = hyperfine.states["7F0"]
excited = hyperfine.states["5D0"]
optical_hyperfine_probabilities = hyperfine.optical_hyperfine_probabilities

In [16]:
def print_table(lists, ndecimals=3, column_size=8):
    for list in lists:
        line = ""
        for kk in list:
            if isinstance(kk, float):
                line += f"{kk:{column_size}.{ndecimals}f}"
            elif isinstance(kk, int):
                line += f"{kk:{column_size}.{ndecimals}f}"
            else:
                line += f"{kk:>{column_size}}"
        print(line)

### Optical hyperfine probabilities

In [17]:
table = [["g\\e"] + excited.states]
for g_state in ground.states:
    row = [g_state]
    for e_state in excited.states:
        row.append(optical_hyperfine_probabilities[g_state][e_state])
    table.append(row)
print("580 nm hyperfine excitation probabilities:")
print_table(table)

580 nm hyperfine excitation probabilities:
     g\e       c      c'       b      b'       a      a'
       a   0.856   0.014   0.031   0.067   0.001   0.031
      a'   0.014   0.856   0.067   0.031   0.031   0.001
       b   0.111   0.004   0.396   0.320   0.040   0.129
      b'   0.004   0.111   0.320   0.396   0.129   0.040
       c   0.005   0.011   0.149   0.036   0.322   0.478
      c'   0.011   0.005   0.036   0.149   0.478   0.322


### Ground M1 matrix elements

In [18]:
M1_elements = ground.m1_elements()
for kk, polarization in enumerate(["σ+", "σ-", "π"]):
    table = [["i\\f"] + ground.states]
    for g_state in ground.states:
        row = [g_state]
        for g_state_1 in ground.states:
            MHz_per_T_to_kHz_per_G = 1e6 / (1e3 / 1e-4)
            row.append(abs(M1_elements[kk][g_state][g_state_1] * MHz_per_T_to_kHz_per_G))
        table.append(row)
    print(f"Polarization: {polarization}. Unit: 2π kHz/G.")
    print_table(table, ndecimals=3)
    print()

Polarization: σ+. Unit: 2π kHz/G.
     i\f       a      a'       b      b'       c      c'
       a   0.502   0.107   0.020   0.021   0.012   0.005
      a'   0.198   0.272   0.198   0.110   0.025   0.030
       b   0.064   0.141   0.305   0.310   0.123   0.043
      b'   0.049   0.200   0.122   0.097   0.280   0.140
       c   0.012   0.106   0.117   0.126   0.165   0.275
      c'   0.008   0.025   0.064   0.161   0.148   0.271

Polarization: σ-. Unit: 2π kHz/G.
     i\f       a      a'       b      b'       c      c'
       a   0.502   0.198   0.064   0.049   0.012   0.008
      a'   0.107   0.272   0.141   0.200   0.106   0.025
       b   0.020   0.198   0.305   0.122   0.117   0.064
      b'   0.021   0.110   0.310   0.097   0.126   0.161
       c   0.012   0.025   0.123   0.280   0.165   0.148
      c'   0.005   0.030   0.043   0.140   0.275   0.271

Polarization: π. Unit: 2π kHz/G.
     i\f       a      a'       b      b'       c      c'
       a   0.265   0.440   0.121   0.072  